<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner<br/>
<hr style='height:2px'>

**Acknowledgements**: This project was organized and facilitated by CS109a faculty members

In [987]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2020-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)

## 1. Obtaining Data

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

We will focus on automating the downloading of each state's data (via ``Requests``).

In [988]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

state_dir = "data/"

In [989]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

1.1: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [5]:
home_page = requests.get(base_url)

In [ ]:
#Status:
home_page.status_code

#Contents:
home_page.content

1.3:
    
In the cell below, we create a new BeautifulSoup object that parses the `home_page` as an HTML document

In [992]:

soup = BeautifulSoup(home_page.text, "html.parser")


1.4:
    
In the cell below, we write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. We use Regular Expressions, in conjunction with BeautifulSoup parsing. Specifically, the goal is to populate the `state_urls` dictionary by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

In [ ]:
state_urls = {}

# I created two lists, one for state names and one for urls, then merged them
state = []
url = []

pattern1 = '(?<=\"/)(.*?)(?=\">)' # To find urls

soup1 = soup.select('th a')

for i in range(len(soup1)):
    state_name = soup1[i].text
    state.append(state_name)
    state_url = re.findall(pattern1,str(soup1[i]))
    url.append('https://usafacts.org/' + state_url[0]) # I added the 'https://usafacts.org/' part to the urls found
       
state_urls = {state[j]: url[j] for j in range(len(state))} # Merging the two lists into one dictionary
state_urls

In [994]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

** 1.4 might be correct


We wish to use the data without having to re-download it every time. So, we save each webpage to our local hard drive. 

1.5:
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. 

In [967]:
#os.mkdir(state_dir) # I used this to create the state_urls directory

for state, url in state_urls.items():
    
    web_page = requests.get(url)
    file = open(state_dir + state, 'w')
    file.write(str(web_page.content))
    
    sleep(2)

## 2. Loading and Exploring Data
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it.

In [15]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

2.1:
    
`load_covid_data()` function:

- Takes as input `state_info`, which is a list of tuples: (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - \# of confirmed cases (total)
    - \# of deaths
    - \# of confirmed cases (per 100k)
- Stores in a **non-PANDAS** data structure named `covid_data` the above 3 pieces of data, **for every county across every state**
- Returns `county_counts`
    <font color='blue'>


In [996]:
def load_covid_data(state_info):
    
    #name of each county
    county = []
    state_names = []
    cases = []
    deaths = []
    cases_per_100k = []
    
    path = [lis[1] for lis in state_info]
    state = [lis[0] for lis in state_info]
    
    for j in range(len(path)):
        
        path[j]
        file = open(path[j],'r')
        Soup = BeautifulSoup(file)
        Soup1 = Soup.select('th a')
        Soup2 = Soup.select ('td')       
        
        for i in range(len(Soup1)):
            county_name = Soup1[i].text
            county.append(county_name)
            state_names.append(state[j])
            
        for i in range(len(Soup2)):
            if i*3+2 <= len(Soup2):
                case = Soup2[i*3].text
                death = Soup2[(i*3)+1].text
                case_per_100k = Soup2[(i*3)+2].text
                cases.append(round(float(case.replace(',',''))))
                deaths.append(round(float(death.replace(',',''))))
                cases_per_100k.append(round(float(case_per_100k.replace(',',''))))   
        
        
        covid_data = {x:(county[x],state_names[x],cases[x],deaths[x],cases_per_100k[x]) for x in range(len(county))}
        print(covid_data)

    return covid_data


In [3]:
#covid_data = load_covid_data(state_info)

2.2:
    
`calculate_county_stats()` function:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people
</div>

In [999]:
def calculate_county_stats(covid_data):

    cases_per_100k = []
    county = []
    state = []

    for i in range(len(covid_data)): #filling the empty sets
        cases_per_100k.append(list(covid_data.values())[i][4])
        county.append(list(covid_data.values())[i][0])
        state.append(list(covid_data.values())[i][1])

    for j in range(len(covid_data)): #finding the lowest   
        if cases_per_100k[j] == min(cases_per_100k):
            print(str(county[j]) + ' \ ' + str(state[j]) + " has the lowest rate of confirmed COVID cases: " + str(cases_per_100k[j]) + " per 100k")

    for k in range(len(covid_data)): #finding the highest     
        if cases_per_100k[k] == max(cases_per_100k):
            print(str(county[k]) + ' \ ' + str(state[k]) + " has the highest rate of confirmed COVID cases: " + str(cases_per_100k[k]) + " per 100k")


In [1000]:
calculate_county_stats(covid_data)

Bristol Bay Borough \ Alaska has the lowest rate of confirmed COVID cases: 0 per 100k
Skagway, Municipality of \ Alaska has the lowest rate of confirmed COVID cases: 0 per 100k
Yakutat, City and Borough of \ Alaska has the lowest rate of confirmed COVID cases: 0 per 100k
Kiowa County \ Colorado has the lowest rate of confirmed COVID cases: 0 per 100k
Kalawao County \ Hawaii has the lowest rate of confirmed COVID cases: 0 per 100k
Carter County \ Montana has the lowest rate of confirmed COVID cases: 0 per 100k
Petroleum County \ Montana has the lowest rate of confirmed COVID cases: 0 per 100k
Blaine County \ Nebraska has the lowest rate of confirmed COVID cases: 0 per 100k
Grant County \ Nebraska has the lowest rate of confirmed COVID cases: 0 per 100k
Hayes County \ Nebraska has the lowest rate of confirmed COVID cases: 0 per 100k
Esmeralda County \ Nevada has the lowest rate of confirmed COVID cases: 0 per 100k
Wheeler County \ Oregon has the lowest rate of confirmed COVID cases: 0 pe

2.3:
    
`calculate_state_deaths()` function:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

</div>

In [1001]:
def calculate_state_deaths(covid_data):
    
    page = requests.get(base_url)
    soup = BeautifulSoup(page.text)
    soup1 = soup.select('th a')
    soup2 = soup.select('td')

    death_state = []
    state = []


    for i in range(len(soup2)): 
        if i*2+1 < len(soup2):
            death_state.append(int(soup2[i*2+1].text.replace(',',''))) # to find death numbers
            state.append(soup1[i].text) # to find the state

    for j in range(len(death_state)):
        if death_state[j] == max(death_state):
            max_death = death_state[j] 
            max_state = state[j] 
        if death_state[j] == min(death_state):
            min_death = death_state[j] 
            min_state = state[j] 

    print(str(min_state) + " has the fewest COVID deaths: " + str(min_death))
    print(str(max_state) + " has the most COVID deaths: " + str(max_death))

In [1002]:
calculate_state_deaths(covid_data)

Alaska has the fewest COVID deaths: 44
New York has the most COVID deaths: 32745


2.4:
    
`calculate_state_deathrate()` function:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

In [1003]:
def calculate_state_deathrate(covid_data):

    state_pop = []
    death_state = []
    death_rate = []

    page = requests.get(base_url)
    soup = BeautifulSoup(page.text)
    soup1 = soup.select('th a')
    soup2 = soup.select('td')


    for i in range(len(soup2)): 
        if i*2+1 < len(soup2):
            death_state.append(int(soup2[i*2+1].text.replace(',',''))) # to find death numbers

    path = [lis[1] for lis in state_info]
    state = [lis[0] for lis in state_info]


    for j in range(len(path)):


        #county_pop.clear()
        #known_cases.clear()
        #cases_per_100k.clear()

        total_pop = 0
        known_cases = []
        cases_per_100k = []
        county_pop = []

        path[j]
        file = open(path[j],'r')
        Soup = BeautifulSoup(file)
        Soup1 = Soup.select('th a')
        Soup2 = Soup.select ('td')

        for i in range(len(Soup2)):
            if i*3+2 <= len(Soup2):
                case = Soup2[i*3].text.replace(',','')
                case_per_100k = Soup2[i*3+2].text.replace(',','')
                known_cases.append(case)
                cases_per_100k.append(case_per_100k)

        for i in known_cases:
            if i == str(0):
                known_cases.remove(i)

        for j in cases_per_100k:
            if j == str(0.0):
                cases_per_100k.remove(j)

        for l in range(len(known_cases)):
            pop = int(known_cases[l]) * 100000 / float(cases_per_100k[l])
            county_pop.append(round(pop)) 

        total_pop = sum(county_pop)
        state_pop.append(total_pop)

    for i in range(len(state_pop)):
        rate = float(death_state[i]) / state_pop[i] 
        death_rate.append(rate)

    for m in range(len(death_rate)):

            if death_rate[m] == max(death_rate):
                max_death = round(1/death_rate[m])
                max_state = state[m] 

            if death_rate[m] == min(death_rate):
                min_death = round(1/death_rate[m]) 
                min_state = state[m] 
    
    print(min_state + " has the lowest COVID death rate; 1 out of every " + str(min_death) + " people has died")
    print(max_state + " has the highest COVID death rate; 1 out of every " + str(max_death) + " people has died")

In [251]:
calculate_state_deathrate(covid_data)

Alaska has the lowest COVID death rate; 1 out of every 16567 people has died
New Jersey has the highest COVID death rate; 1 out of every 554 people has died


## 3. PANDAS
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

3.1:

In #2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

`convert_to_pandas()` function, converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 5 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# covid cases per 100k (Integer)
    - \# covid deaths (Integer)
</div>

In [750]:
def convert_to_pandas(covid_data):
    
    covid_df = pd.DataFrame.from_dict(covid_data, orient='index', columns = ['county','state','# total covid cases', '# covid deaths', '# covid cases per 100k'])
    return covid_df

In [1004]:
covid_df = convert_to_pandas(covid_data)
covid_df

,county,state,# total covid cases,# covid deaths,# covid cases per 100k
0,Autauga County,Alabama,1585,24,2837
1,Baldwin County,Alabama,4978,47,2230
2,Barbour County,Alabama,801,7,3245
3,Bibb County,Alabama,610,9,2724
4,Blount County,Alabama,1464,13,2532
...,...,...,...,...,...
3137,Sweetwater County,Wyoming,317,2,749
3138,Teton County,Wyoming,476,1,2029
3139,Uinta County,Wyoming,312,2,1543
3140,Washakie County,Wyoming,111,6,1422


3.2:

`calculate_county_stats2()` function:

1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

</div>

In [1005]:
def calculate_county_stats2(covid_df):
    
    df = covid_df.sort_values(by='# covid cases per 100k', ascending = True).head(10)
    df1 = covid_df.sort_values(by='# covid cases per 100k', ascending = False).head(10)
    min_county = df.iloc[0:9]['county']
    min_state =  df.iloc[0:9]['state']
    min_death =  df.iloc[0:9]['# covid cases per 100k']
    max_county = df1.iloc[0]['county']
    max_state = df1.iloc[0]['state']
    max_death = df1.iloc[0]['# covid cases per 100k']
    
    print(min_county + " (" + min_state + ") has the lowest rate of confirmed COVID cases: " + str(min_death) + " per 100k")
    print(max_county + " (" + max_state + ") has the highest rate of confirmed COVID cases: " + str(max_death) + " per 100k")

In [1006]:
calculate_county_stats2(covid_df)

71      Bristol Bay Borough (Alaska) has the lowest ra...
1603    Carter County (Montana) has the lowest rate of...
90      Skagway, Municipality of (Alaska) has the lowe...
2242    Wheeler County (Oregon) has the lowest rate of...
2673    Loving County (Texas) has the lowest rate of c...
94      Yakutat, City and Borough of (Alaska) has the ...
2657    King County (Texas) has the lowest rate of con...
548     Kalawao County (Hawaii) has the lowest rate of...
1658    Blaine County (Nebraska) has the lowest rate o...
dtype: object
Trousdale County (Tennessee) has the highest rate of confirmed COVID cases: 14543 per 100k


3.3:
    
`calculate_state_deaths2()` function: 

1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

In [1007]:
def calculate_state_deaths2(covid_df):
    
    df = covid_df.groupby(by ="state").sum()
    df1 = df.sort_values('# covid deaths', ascending = False).head(10)
    df2 = df.sort_values('# covid deaths', ascending = True).head(10)

    max_deaths = df1.iloc[0]['# covid deaths']
    min_deaths = df2.iloc[0]['# covid deaths']

    # By printing df1 and df2 I could see that max_state = New York and min_state = Alaska
    max_state = 'New York' 
    min_state = 'Alaska'

    print(min_state + " has the fewest COVID deaths: " + str(min_deaths))
    print(max_state + " has the most COVID deaths: " + str(max_deaths))
    
    

In [1008]:
calculate_state_deaths2(covid_df)

Alaska has the fewest COVID deaths: 44
New York has the most COVID deaths: 32623


3.4:
    
`calculate_state_deathrate2()` function:

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

In [1009]:
def calculate_state_deathrate2(covid_df):
    
    df = covid_df.groupby(by ="state").sum()

    a = covid_df['# total covid cases']
    b = a * 100000
    c = covid_df['# covid cases per 100k']
    d = b / c

    covid_df['population'] = d

    df2 = covid_df.groupby(by='state').sum()
    number = df2['# covid deaths']/df2['population']

    df2['number'] = number

    df3 = df2.sort_values(by="number",ascending=False)
    high_rate = int(round(1 / df3.iloc[0]['number']))

    df4 = df2.sort_values(by="number",ascending=True)
    low_rate = int(round(1 / df4.iloc[0]['number']))

    #I looked at the names of the states from the table

    print('Alaska' + " has the lowest COVID death rate; 1 out of every " + str(low_rate) + " people has died")
    print('New Jersey' + " has the highest COVID death rate; 1 out of every " + str(high_rate) + " people has died")

In [1010]:
calculate_state_deathrate2(covid_df)

Alaska has the lowest COVID death rate; 1 out of every 16567 people has died
New Jersey has the highest COVID death rate; 1 out of every 554 people has died


These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

3.5:
    
`add_death_stats()` function:

- `case_fatality_rate` and
- `# covid deaths per 100k`

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# total covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate

In [1011]:
def add_death_stats(covid_df):
    
#covid_df = covid_df.drop(columns =['number']) #I used this to drop the column I added in the previous part

    covid_df['# covid deaths per 100k'] = (covid_df['# covid deaths']*100000) / covid_df['population']
    covid_df['case_fatality_rate'] = covid_df['# covid deaths'] / covid_df['# total covid cases']
    covid_df = covid_df.sort_values('case_fatality_rate', ascending = True)
    
    return covid_df

In [1012]:
covid_updated = add_death_stats(covid_df)
covid_updated

,county,state,# total covid cases,# covid deaths,# covid cases per 100k,population,# covid deaths per 100k,case_fatality_rate
3141,Weston County,Wyoming,23,0,332,6927.710843,0.0,0.0
1759,Pershing County,Nevada,21,0,312,6730.769231,0.0,0.0
2793,Rich County,Utah,13,0,524,2480.916031,0.0,0.0
1760,Storey County,Nevada,9,0,218,4128.440367,0.0,0.0
2792,Piute County,Utah,6,0,406,1477.832512,0.0,0.0
...,...,...,...,...,...,...,...,...
1751,Esmeralda County,Nevada,0,0,0,NaN,NaN,NaN
2242,Wheeler County,Oregon,0,0,0,NaN,NaN,NaN
2539,Borden County,Texas,0,0,0,NaN,NaN,NaN
2657,King County,Texas,0,0,0,NaN,NaN,NaN


3.6:
    
some trends we've noticed:

In [ ]:
The bigger the population, the more covid cases there are (obviously), but the number of covid cases per 100k 
is almost the same across all counties (with the exception of some states like Alaska), i.e. on the same order of magnitude
it is also worth noting that according to this data, the fatality_rate is very low (it's still bad of course), but I was
surprised to see those figures!

In [ ]:
Following up on my previous answer, no I don't think it is very reliable. I find it hard to believe that many counties
have zero covid-related deaths. So, I suspect that either the samples they are taking into consideration vary in size
from a state to state, or that there are many covid-related deaths that are being disregarded (especially that we are talking
about populations exceeding tens of thousands per county!)

In [ ]:
by checking the ratio of covid-deaths to covid cases for a few counties, I could see that it is roughly equal to 0.015
which means tha there are roughly 1000 cases for 15 deaths. This is a high number of cases for a county, so I would suspect
that it is either in New York or New Jersey (because they have some of the highest covid cases numbers)

In [ ]:
1- How does covid affect people from different socioeconomic backgrounds? (on the one hand, wealthy people 
are able to get tested more often and thus we excpect the number of cases to be high, but wealthy people also live
healthier lifestyles which makes them less vulnerable to the virus)

2- Does the virus mutate? if so, how fast? What percentage of the confirmed cases aquired the virus a second time?

3- Are masks necessary when walking outside? (I'm not an anti-masker I swear, but to what extent do masks help?)


## 4. MORE DATA
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2016_by_county.csv`.

4.1:

`merge_data()` function:

1. First, load `election2016_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, we merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 7 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- case_fatality_rate

along with these 14 columns from `election2016_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- votergap
- trump
- clinton

**NOTES:**
- We are dropping two columns from `election2016_by_county.csv`:
    - fipscode
    - population

In [1013]:
def merge_data(covid_updated, filepath):

    df = pd.read_csv('Desktop/election2016_by_county.csv')
    df2 = df.drop(columns = ['fipscode','population'])
    merged_df = df2.merge(covid_updated)
    return merged_df
merged = merge_data(covid_updated, 'Desktop/election2016_by_county.csv')
merged

,state,county,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,...,cancer,votergap,trump,clinton,# total covid cases,# covid deaths,# covid cases per 100k,population,# covid deaths per 100k,case_fatality_rate
0,Alabama,Autauga County,2.842,22.733,51.475,5.2,54366,13.8,21.9,28.6,...,186.5,49.479,73.436,23.957,1585,24,2837,55868.875573,42.957729,0.015142
1,Alabama,Baldwin County,4.550,12.934,51.261,5.5,49626,11.0,28.6,22.3,...,229.4,57.786,77.351,19.565,4978,47,2230,223228.699552,21.054640,0.009442
2,Alabama,Barbour County,4.424,49.885,46.589,8.9,34971,25.4,13.6,31.8,...,223.3,5.611,52.271,46.660,801,7,3245,24684.129430,28.358302,0.008739
3,Alabama,Bibb County,2.409,23.930,46.110,6.6,39546,22.1,10.2,33.9,...,230.3,55.544,76.966,21.422,610,9,2724,22393.538913,40.190164,0.014754
4,Alabama,Blount County,8.954,4.229,50.592,5.4,45567,21.9,12.3,28.0,...,205.3,81.382,89.852,8.470,1464,13,2532,57819.905213,22.483607,0.008880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3074,Wyoming,Sweetwater County,15.829,5.846,48.145,4.6,72604,9.5,18.1,26.0,...,110.0,53.538,72.943,19.405,317,2,749,42323.097463,4.725552,0.006309
3075,Wyoming,Teton County,15.018,4.778,48.368,3.8,75348,4.3,51.9,10.8,...,104.7,-27.866,32.194,60.061,476,1,2029,23459.832430,4.262605,0.002101
3076,Wyoming,Uinta County,9.072,4.601,49.702,4.9,56800,10.8,18.7,21.5,...,117.9,61.493,76.419,14.926,312,2,1543,20220.349968,9.891026,0.006410
3077,Wyoming,Washakie County,14.337,5.259,49.075,4.0,50802,10.9,21.2,24.4,...,234.2,64.038,78.358,14.320,111,6,1422,7805.907173,76.864865,0.054054


In [908]:
merged = merge_data(covid_updated, 'election2016_by_county.csv')

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

4.2:

4.2:
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`?
</div>

In [1016]:
a = []
b = []

for i in covid_updated['county']:
    a.append(i)
    
for j in merged['county']:
    b.append(j)

print(str(len(a)-len(b)) + ' rows')

63 rows


4.2:

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. 

In [1017]:
a = []
b = []

for i in covid_updated['county']:
    a.append(i)
    
for j in merged['county']:
    b.append(j)

for i in range(len(a)):
    for j in range(len(b)):
        if i < len(b):
            if a[i] != b[j]:
                #print(a[i]) #I used this code to check three counties
print('Three counties are: Weston County (Wyoming) -- Iron County (Michigan) -- Deer Lodge County (Montana)')

Three counties are: Weston County (Wyoming) -- Iron County (Michigan) -- Deer Lodge County (Montana)


Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

4.3:

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information.

In [1018]:

merged[(merged != 0).all(1)]


,state,county,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,...,cancer,votergap,trump,clinton,# total covid cases,# covid deaths,# covid cases per 100k,population,# covid deaths per 100k,case_fatality_rate
0,Alabama,Autauga County,2.842,22.733,51.475,5.2,54366,13.8,21.9,28.6,...,186.5,49.479,73.436,23.957,1585,24,2837,55868.875573,42.957729,0.015142
1,Alabama,Baldwin County,4.550,12.934,51.261,5.5,49626,11.0,28.6,22.3,...,229.4,57.786,77.351,19.565,4978,47,2230,223228.699552,21.054640,0.009442
2,Alabama,Barbour County,4.424,49.885,46.589,8.9,34971,25.4,13.6,31.8,...,223.3,5.611,52.271,46.660,801,7,3245,24684.129430,28.358302,0.008739
3,Alabama,Bibb County,2.409,23.930,46.110,6.6,39546,22.1,10.2,33.9,...,230.3,55.544,76.966,21.422,610,9,2724,22393.538913,40.190164,0.014754
4,Alabama,Blount County,8.954,4.229,50.592,5.4,45567,21.9,12.3,28.0,...,205.3,81.382,89.852,8.470,1464,13,2532,57819.905213,22.483607,0.008880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,Wyoming,Sublette County,7.607,5.394,46.247,5.1,77222,5.3,23.6,16.5,...,106.4,64.347,79.334,14.987,54,1,549,9836.065574,10.166667,0.018519
3074,Wyoming,Sweetwater County,15.829,5.846,48.145,4.6,72604,9.5,18.1,26.0,...,110.0,53.538,72.943,19.405,317,2,749,42323.097463,4.725552,0.006309
3075,Wyoming,Teton County,15.018,4.778,48.368,3.8,75348,4.3,51.9,10.8,...,104.7,-27.866,32.194,60.061,476,1,2029,23459.832430,4.262605,0.002101
3076,Wyoming,Uinta County,9.072,4.601,49.702,4.9,56800,10.8,18.7,21.5,...,117.9,61.493,76.419,14.926,312,2,1543,20220.349968,9.891026,0.006410


Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [1019]:
merged.describe()

,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton,# total covid cases,# covid deaths,# covid cases per 100k,population,# covid deaths per 100k,case_fatality_rate
count,3079.000000,3079.000000,3079.000000,3079.000000,3079.000000,3079.000000,3079.000000,3079.000000,3079.000000,3079.000000,3029.000000,3065.000000,3065.000000,3065.000000,3079.000000,3079.000000,3079.000000,3.067000e+03,3067.000000,3067.000000
mean,9.182587,14.697902,49.902878,5.518090,46994.699253,15.039006,19.928743,26.040403,31.064696,226.756902,228.515087,32.608732,63.960641,31.351915,2068.247158,61.976291,1634.898344,1.049925e+05,38.008947,0.021768
std,13.631337,15.876698,2.254409,1.977942,11879.653521,6.765832,8.716121,5.185448,4.493611,1685.359070,55.782261,30.231787,15.318398,15.043041,8624.958470,321.457909,1361.694245,3.362762e+05,49.078625,0.023367
min,0.205000,0.855000,28.479000,1.800000,21658.000000,1.300000,2.600000,8.100000,11.800000,0.100000,46.200000,-88.725000,4.122000,3.145000,0.000000,0.000000,0.000000,4.039238e+02,0.000000,0.000000
25%,2.113000,4.141000,49.473000,4.200000,38887.000000,9.900000,13.900000,22.700000,28.400000,16.900000,193.500000,15.825000,55.490000,20.368000,104.000000,1.000000,712.500000,1.112840e+04,5.953962,0.006274
50%,3.994000,8.069000,50.356000,5.300000,45211.000000,13.600000,17.800000,25.900000,31.200000,44.400000,230.200000,38.687000,66.892000,28.237000,352.000000,6.000000,1291.000000,2.617587e+04,21.363636,0.016043
75%,9.388500,19.318500,51.030000,6.500000,52462.000000,19.300000,23.500000,29.500000,33.900000,109.050000,264.800000,54.936000,75.229000,39.544000,1127.500000,25.000000,2192.500000,6.809709e+04,50.222406,0.029703
max,95.824000,94.531000,56.739000,24.000000,122641.000000,53.300000,72.000000,41.400000,47.600000,69468.400000,458.300000,91.636000,95.273000,92.847000,254656.000000,7306.000000,14543.000000,1.003768e+07,484.860158,0.333333


Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

4.4:
    
`partition_df()` function:
inputs:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

In [1020]:
def partition_df(df, column_name, min, max):
    
    partition = merged.loc[(merged[column_name] >= min)&(merged[column_name] <= max)]
    return partition

4.5:

In [1021]:

partition_df(merged, 'income', 21658, 45211).describe()


#I have noticed that the higher the income, the higher the total number of covid cases. Originally I thought this makes
#sense because the higher your income, the better chances you have at getting testes, but I was surprised to see that
#the number of deaths per 100k is also higher for this group of people. I'm not sure how to explain this, either my reasoning
#is not valid, or there is some explanation that I might be missing.

#partition_df(merged, 'your feature here', your_min_value, your_max_va).describe()

,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton,# total covid cases,# covid deaths,# covid cases per 100k,population,# covid deaths per 100k,case_fatality_rate
count,1540.000000,1540.000000,1540.000000,1540.000000,1540.000000,1540.000000,1540.000000,1540.000000,1540.000000,1540.000000,1518.000000,1540.000000,1540.000000,1540.000000,1540.000000,1540.000000,1540.00000,1.535000e+03,1535.000000,1535.000000
mean,8.839739,17.997643,49.897631,6.419026,38269.012987,18.410519,15.826104,28.425065,32.512987,115.536818,235.945652,35.444724,65.801910,30.357193,1096.695455,30.618182,1870.30974,4.881290e+04,45.804267,0.022326
std,15.428433,18.688267,2.588463,2.025101,4672.780733,6.602018,5.594135,4.793534,4.347585,293.166986,52.786507,30.030224,15.091555,15.038313,5101.198164,182.830996,1578.56974,1.338936e+05,56.401799,0.022498
min,0.205000,0.937000,28.479000,2.100000,21658.000000,1.300000,5.800000,13.600000,16.500000,0.300000,82.100000,-79.134000,8.322000,4.749000,0.000000,0.000000,0.00000,4.039238e+02,0.000000,0.000000
25%,1.807000,4.340500,49.507250,5.200000,35053.500000,13.400000,12.200000,25.200000,29.800000,19.175000,203.500000,20.759250,58.166000,19.672500,101.000000,1.000000,709.75000,1.024110e+04,6.644791,0.007143
50%,3.111000,9.135000,50.482000,6.200000,38887.000000,18.000000,14.700000,28.500000,32.600000,42.350000,236.250000,43.559000,69.501500,26.262500,309.500000,5.000000,1469.50000,2.047073e+04,26.530825,0.017241
75%,7.168250,27.728000,51.221500,7.500000,42001.500000,22.600000,18.300000,32.000000,35.300000,92.225000,268.175000,56.722250,76.497000,37.658750,782.500000,18.000000,2697.25000,4.068570e+04,64.874510,0.030938
max,95.824000,93.411000,56.739000,24.000000,45211.000000,53.300000,48.800000,41.400000,47.600000,5157.500000,458.300000,86.453000,92.097000,88.727000,164299.000000,4936.000000,14543.00000,2.717033e+06,484.860158,0.333333


`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

In [1022]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

,state,county,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,...,votergap,trump,clinton,# total covid cases,# covid deaths,# covid cases per 100k,population,# covid deaths per 100k,case_fatality_rate,income group
0,Alabama,Autauga County,2.842,22.733,51.475,5.2,54366,13.8,21.9,28.6,...,49.479,73.436,23.957,1585,24,2837,55868.875573,42.957729,0.015142,income-group-4
1,Alabama,Baldwin County,4.550,12.934,51.261,5.5,49626,11.0,28.6,22.3,...,57.786,77.351,19.565,4978,47,2230,223228.699552,21.054640,0.009442,income-group-3
2,Alabama,Barbour County,4.424,49.885,46.589,8.9,34971,25.4,13.6,31.8,...,5.611,52.271,46.660,801,7,3245,24684.129430,28.358302,0.008739,income-group-1
3,Alabama,Bibb County,2.409,23.930,46.110,6.6,39546,22.1,10.2,33.9,...,55.544,76.966,21.422,610,9,2724,22393.538913,40.190164,0.014754,income-group-2
4,Alabama,Blount County,8.954,4.229,50.592,5.4,45567,21.9,12.3,28.0,...,81.382,89.852,8.470,1464,13,2532,57819.905213,22.483607,0.008880,income-group-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3074,Wyoming,Sweetwater County,15.829,5.846,48.145,4.6,72604,9.5,18.1,26.0,...,53.538,72.943,19.405,317,2,749,42323.097463,4.725552,0.006309,income-group-4
3075,Wyoming,Teton County,15.018,4.778,48.368,3.8,75348,4.3,51.9,10.8,...,-27.866,32.194,60.061,476,1,2029,23459.832430,4.262605,0.002101,income-group-4
3076,Wyoming,Uinta County,9.072,4.601,49.702,4.9,56800,10.8,18.7,21.5,...,61.493,76.419,14.926,312,2,1543,20220.349968,9.891026,0.006410,income-group-4
3077,Wyoming,Washakie County,14.337,5.259,49.075,4.0,50802,10.9,21.2,24.4,...,64.038,78.358,14.320,111,6,1422,7805.907173,76.864865,0.054054,income-group-3


4.6:

Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [1023]:

new_df = merged.groupby('income group').mean()
new_df


,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton,# total covid cases,# covid deaths,# covid cases per 100k,population,# covid deaths per 100k,case_fatality_rate
income group,,,,,,,,,,,,,,,,,,,,
income-group-1,8.954673,24.808803,49.972505,7.318502,33902.058737,22.047430,13.603671,30.212041,33.792952,90.745521,237.447788,29.844943,63.341623,33.496659,769.558003,26.753304,2345.296623,28287.206309,62.429867,0.024324
income-group-2,8.792035,12.771166,49.839881,5.730927,41617.028881,15.631167,17.519254,27.055716,31.553550,132.004332,235.065025,39.917444,67.788922,27.871510,1358.498195,33.407942,1510.806258,64534.270918,32.880828,0.020598
income-group-3,8.785552,10.812178,49.848097,4.888181,48456.864542,12.864807,20.375697,25.175432,30.692961,328.459894,227.955420,35.377553,65.024052,29.646486,1985.345286,48.337317,1453.484728,98247.321591,27.663717,0.019709
income-group-4,10.139252,11.800534,49.959617,4.377273,62085.512285,10.582432,27.266708,22.314005,28.627027,343.195209,214.829838,24.816137,59.537451,34.721333,3956.003686,133.226044,1335.074939,216895.368572,32.358578,0.022730


4.7:

In [2]:
It is interesting, because now there does not seem to be much correlation between Income and covid deaths per 100k
there are sligh differences of course, but nothing too major, although I find it hard to believe that this is 
a valid conclusion..

SyntaxError: invalid syntax (<ipython-input-2-d5781417d513>, line 1)

4.7: 
Some weaknesses from this view of the data:

In [ ]:
Information is very limited. It is hard to find any correlations because now all values seem to be almost equal
(withing one column), and so it is hard to check the validity of any conclusion based on this table